In [16]:
# You might need to install oddt:    conda install -c oddt oddt

from oddt import metrics
from sklearn.metrics import average_precision_score as avp
from sklearn.metrics import plot_precision_recall_curve, roc_curve

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdMolAlign
from rdkit.Chem import rdMolDescriptors

from copy import deepcopy
from espsim import GetShapeSim, GetEspSim
from espsim.helpers import mlCharges
from joblib import Parallel,  delayed

import urllib.request
import pandas as pd

In [22]:
def align(prbMol, refMol, prbCrippen, refCrippen, i, j):
    alignment = rdMolAlign.GetCrippenO3A(prbMol, refMol, prbCrippen, refCrippen, i, j)
    alignment.Align()

def get_esp(prbMol,refMol,i,j,partialCharges,prbCharge=None,refCharge=None,renormalize = True,metric=True):
    try:
        if partialCharges=='ml':
            esp=GetEspSim(prbMol,refMol,i,j,prbCharge=prbCharge,refCharge=refCharge,renormalize = renormalize,metric = metric)
        else:
            esp=GetEspSim(prbMol,refMol,i,j,partialCharges = partialCharges,renormalize = renormalize,metric = metric)
    except:
        esp=0
    return esp

def embed(mol,n=10):
    if mol is not None:
        ps = AllChem.ETKDGv2()
        ps.useRandomCoords=False
        AllChem.EmbedMultipleConfs(mol, n, ps)
        if mol.GetNumConformers() != n:
            ps.useRandomCoords=True
            AllChem.EmbedMultipleConfs(mol, n, ps)
        if mol.GetNumConformers() != n:
            print("could not embed")
            return None
    return mol

def AlignScore(prbMol,refMol,prbCharge,refCharge):
    #Need to copy mols, otherwise the alignment will change the positions of the molecule object permanently                                                                                                                               
    prbMol=deepcopy(prbMol)
    refMol=deepcopy(refMol)

    #Align molecules                                                                                                                                                                                                                       
    prbCrippen = rdMolDescriptors._CalcCrippenContribs(prbMol)
    refCrippen = rdMolDescriptors._CalcCrippenContribs(refMol)
    align(prbMol, refMol, prbCrippen, refCrippen, 0, 0)

    #Calculate shape and esp similarities  
    shape = GetShapeSim(prbMol,refMol)
    esp=get_esp(prbMol,refMol,0,0,'mmff',metric='tanimoto')
    esp_gasteiger=get_esp(prbMol,refMol,0,0,'gasteiger',metric='tanimoto')
    esp_ml=get_esp(prbMol,refMol,0,0,'ml',prbCharge=prbCharge,refCharge=refCharge,metric='tanimoto')
        
    return shape+esp,shape+esp_gasteiger, shape+esp_ml

def EmbedAlignScore(prbMol,refMol,prbCharge,refCharge,best='shape'):
    #Need to copy mols, otherwise the alignment will change the positions of the molecule object permanently                                                                                                                               
    prbMol=deepcopy(prbMol)
    refMol=deepcopy(refMol)

    prbCrippen = rdMolDescriptors._CalcCrippenContribs(prbMol)
    refCrippen = rdMolDescriptors._CalcCrippenContribs(refMol)

    best_score=0
    best_i=0
    for i in range(prbMol.GetNumConformers()):
        align(prbMol, refMol, prbCrippen, refCrippen, i, 0)
        shape = GetShapeSim(prbMol,refMol,i,0)
        if shape>best_score:
            best_score=shape
            best_i=i
    align(prbMol, refMol, prbCrippen, refCrippen, best_i, 0)
    shape = GetShapeSim(prbMol,refMol,best_i,0)
    esp=get_esp(prbMol,refMol,best_i,0,'mmff',metric='tanimoto')
    esp_gasteiger=get_esp(prbMol,refMol,best_i,0,'gasteiger',metric='tanimoto')
    esp_ml=get_esp(prbMol,refMol,best_i,0,'ml',prbCharge=prbCharge,refCharge=refCharge,metric='tanimoto')

    return shape+esp,shape+esp_gasteiger, shape+esp_ml

In [19]:
#In the following, we only explore one out of the 102 DUD-E targets to make this notebook fast to execute.
#We recommend running the full benchmark (all 102 targets) on a cluster, since especially the embedding is slow.

In [29]:
systems=['Shape+ESP Sim MMFF','Shape+ESP Sim Gasteiger','Shape+ESP Sim ML']

for name in ['ampc']: #Only one target, add more if you want to explore other DUD-E targets
    print("Download actives, decoys and ligand")
    url = 'http://dude.docking.org/targets/'+name+'/actives_final.sdf.gz'
    filename = 'data_benchmark_4/'+name+'_actives.sdf.gz'
    urllib.request.urlretrieve(url, filename)
    url = 'http://dude.docking.org/targets/'+name+'/decoys_final.sdf.gz'
    filename = 'data_benchmark_4/'+name+'_decoys.sdf.gz'
    urllib.request.urlretrieve(url, filename)
    url = 'http://dude.docking.org/targets/'+name+'/crystal_ligand.mol2'
    filename = 'data_benchmark_4/'+name+'_ligand.mol2'
    urllib.request.urlretrieve(url, filename)

    print("Reading in files")
    inf = gzip.open('data_benchmark_4/'+name+'_actives.sdf.gz')
    gzsuppl= Chem.ForwardSDMolSupplier(inf)
    mols = [Chem.AddHs(mol,addCoords=True) for mol in gzsuppl if mol is not None]
    mol_names = [mol.GetProp("_Name") for mol in mols]
    inf.close()
    inf = gzip.open('data_benchmark_4/'+name+'_decoys.sdf.gz')
    gzsuppl= Chem.ForwardSDMolSupplier(inf)
    decoys = [Chem.AddHs(mol,addCoords=True) for mol in gzsuppl if mol is not None]
    decoy_names=[ mol.GetProp("_Name") for mol in decoys]
    inf.close()  
    
    print("Embedding molecules")
    mols = Parallel(n_jobs=8, verbose=1)(delayed(embed)(mol,n=10) for mol in mols)
    mol_names = [mol_names[i] for i in range(len(mols)) if mols[i] is not None]
    mols = [mol for mol in mols if mol is not None]       
    decoys = Parallel(n_jobs=8, verbose=1)(delayed(embed)(mol,n=10) for mol in decoys)
    decoys_names = [decoy_names[i] for i in range(len(decoys)) if decoys[i] is not None]
    decoys = [mol for mol in decoys if mol is not None]  

    pdb = Chem.MolFromMol2File('data_benchmark_4/'+name+'_ligand.mol2')
    pdb = Chem.AddHs(pdb,addCoords=True)

    print("Preparing ML charges")
    prbCharges=mlCharges(mols+decoys)
    refCharge=mlCharges([pdb])[0]
    
    print("Computing similarities for 10 poses")
    df = pd.DataFrame(list(zip(mols+decoys, [True]*len(mols)+[False]*len(decoys),mol_names+decoys_names)),columns=['mol','active','name'])
    print(name,len(df))
    results = Parallel(n_jobs=8, verbose=1)(delayed(EmbedAlignScore)(m,pdb,prbCharges[i],refCharge) for i,m in enumerate(df['mol']))
    for i in range(len(systems)):
        df['Random poses (10) '+systems[i]] = [x[i] for x in results]
        
    results2 = []
    all_systems=[x+systems[i] for i in range(len(systems)) for x in ['Random poses (10) ']]

    for col in all_systems:
        strd = df.sort_values(col, ascending=False).drop_duplicates('name')
        strd = strd.fillna(0)

        fpr, tpr, thresholds= roc_curve(strd['active'], strd[col])

        bedroc = metrics.bedroc(strd['active'].values, strd[col].values)
        rie = metrics.rie(strd['active'].values, strd[col].values)
        roc_log_auc = metrics.roc_log_auc(strd['active'].values, strd[col].values, ascending_score=False)
        roc_auc = metrics.roc_auc(strd['active'].values, strd[col].values, ascending_score=False)
        ef1=metrics.enrichment_factor(strd['active'].values, strd[col].values)
        ap = avp(strd['active'],  strd[col])

        results2.append([col, bedroc, rie, roc_log_auc, roc_auc, ap, ef1])

    results_df = pd.DataFrame(data=results2, columns=['Algorithm', 'BEDROC', 'RIE', 'log ROC AUC', 'ROC AUC', 'Average Precision', 'EF1'])
    results_df.set_index('Algorithm').style.highlight_max(color = 'lightgreen', axis = 0)
    
    print(results_df)

    results_df.to_csv("data_benchmark_4/results_"+name+".csv")

Download actives, decoys and ligand
Reading in files
Embedding molecules


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  47 out of  62 | elapsed:    1.5s remaining:    0.5s
[Parallel(n_jobs=8)]: Done  62 out of  62 | elapsed:    2.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  52 tasks      | elapsed:    2.3s
[Parallel(n_jobs=8)]: Done 255 tasks      | elapsed:   14.9s
[Parallel(n_jobs=8)]: Done 505 tasks      | elapsed:   23.1s
[Parallel(n_jobs=8)]: Done 855 tasks      | elapsed:   47.0s
[Parallel(n_jobs=8)]: Done 1305 tasks      | elapsed:  1.4min
[Parallel(n_jobs=8)]: Done 1855 tasks      | elapsed:  2.0min
[Parallel(n_jobs=8)]: Done 2505 tasks      | elapsed:  2.4min
[Parallel(n_jobs=8)]: Done 2887 out of 2902 | elapsed:  2.6min remaining:    0.8s
[Parallel(n_jobs=8)]: Done 2902 out of 2902 | elapsed:  6.4min finished


Preparing ML charges
Computing similarities for 10 poses
ampc 2963


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  52 tasks      | elapsed:    1.7s
[Parallel(n_jobs=8)]: Done 352 tasks      | elapsed:   11.4s
[Parallel(n_jobs=8)]: Done 852 tasks      | elapsed:   26.7s
[Parallel(n_jobs=8)]: Done 1552 tasks      | elapsed:   53.6s
[Parallel(n_jobs=8)]: Done 2452 tasks      | elapsed:  1.4min
[Parallel(n_jobs=8)]: Done 2948 out of 2963 | elapsed:  1.6min remaining:    0.5s
[Parallel(n_jobs=8)]: Done 2963 out of 2963 | elapsed:  1.6min finished


                                   Algorithm    BEDROC       RIE  log ROC AUC  \
0       Random poses (10) Shape+ESP Sim MMFF  0.185114  3.148272     0.277613   
1  Random poses (10) Shape+ESP Sim Gasteiger  0.175046  2.977034     0.271265   
2         Random poses (10) Shape+ESP Sim ML  0.471529  8.019384     0.481803   

    ROC AUC  Average Precision        EF1  
0  0.729792           0.093399   6.204741  
1  0.724862           0.084930   6.204741  
2  0.806385           0.300807  31.023707  
